# Title: CORRELATION MODEL IN THE ADOPTION OF E-PAYMENT SERVICES

## Load Libraries

In [1]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import ipywidgets as ipw
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from sklearn import svm, tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, matthews_corrcoef, precision_recall_curve, roc_curve

import scipy.stats as stats
from scipy.stats import pearsonr, spearmanr, kendalltau, pointbiserialr, chi2, chi2_contingency

from xgboost import XGBClassifier

from dtreeviz.trees import *

%matplotlib inline

## Load Custom Made Libraries

In [2]:
from Utilities.CFS import *
from Utilities.reliefF import *
from Utilities.accuracy import *
from Utilities.corr_matrix import *
from Utilities.forward_selection import *
from Utilities.backward_elimination import *
from Utilities.prob_distributed_feature import *

from Visualization.model_graph import *
from Visualization.network_graph import *

from skfeature.utility.mutual_information import *

## Load Dataset

In [3]:
df = pd.read_csv("Dataset/E-payment Cryptocurrency Coin.csv")

## Label Binarizer

In [4]:
def convert_nominal(arr, term_arr):
    tmp_dict = {val:ind for (ind, val) in enumerate(term_arr)}
    return arr.map(lambda x : tmp_dict[x])

## Split Dataset

In [5]:
index_dict = {
    "Age": ["< 25 years", "26 - 40 years", "41 - 55 years", "above 55 years"],
    "Gender": ["Male", "Female"],
    "Marital Status": ["Single", "Married", "Other"],
    "Education Level": ['Primary school', 'Secondary/High school', 'College/university', 'Graduate school', 'Other'],
    "Work Industry": ['Banking / Finance', 'Education', 'Healthcare', 'Manufacturing', 'Retail / Hypermarket', 'Other'],
    "Work Position": ['Junior management', 'Middle management', 'Top management', 'Professional', 'Other']
}

In [6]:
mod_fac_df = pd.DataFrame()

column_arr = ["Age", "Gender", "Marital Status", "Education Level", "Work Industry", "Work Position"]

for (ind, col_name) in enumerate(column_arr):
    mod_fac_df[col_name] = df.iloc[:, 6 + ind]
    
# Replace Values in Work Industry
mod_fac_df = mod_fac_df.replace("Baking / Finance", "Banking / Finance")

# Label Binarize all columns
for col in column_arr:
    mod_fac_df[col] = convert_nominal(mod_fac_df[col], index_dict[col])

In [7]:
utaut_fac_df = pd.DataFrame()

column_arr = df.iloc[:, 28:].columns
column_arr = [col.split(": ")[0] for col in column_arr]

for (ind, col_name) in enumerate(column_arr):
    utaut_fac_df[col_name] = df.iloc[:, 28 + ind]
    utaut_fac_df[col_name] = utaut_fac_df[col_name].map(lambda x : x - 1)
    
# Change Data Type to int
utaut_fac_df = utaut_fac_df.astype(int)

In [8]:
tmp_mod_fac_df = pd.DataFrame()

column_arr = ["Age", "Gender", "Marital Status", "Education Level", "Work Industry", "Work Position"]

for (ind, col_name) in enumerate(column_arr):
    tmp_mod_fac_df[col_name] = df.iloc[:, 6 + ind]
    
tmp_mod_fac_df = tmp_mod_fac_df.replace("Baking / Finance", "Banking / Finance")

all_df_arr = [tmp_mod_fac_df, utaut_fac_df, df.iloc[:, [17, 18]]]

all_df = pd.concat(all_df_arr, axis = 1)
all_df.columns = all_df.columns.tolist()[:-2] + [col.split("[")[1][4:-1] for col in all_df.columns[-2:]]

In [9]:
all_df

,Age,Gender,Marital Status,Education Level,Work Industry,Work Position,PE1,PE2,PE3,PE4,...,T1,T2,T3,T4,BI1,BI2,BI3,BI4,Have you ever purchased anything using the E-payment mode?,"In the next six months, do you plan to purchase anything using the E-payment mode?"
0,< 25 years,Female,Single,College/university,Banking / Finance,Other,1,1,1,1,...,1,0,1,2,2,2,2,2,Yes,Yes
1,< 25 years,Female,Single,College/university,Other,Other,0,0,0,0,...,1,1,1,1,1,1,1,1,Yes,Yes
2,41 - 55 years,Female,Single,College/university,Manufacturing,Middle management,3,4,3,4,...,2,2,2,3,2,3,3,3,Yes,Yes
3,< 25 years,Male,Single,College/university,Education,Other,2,2,2,2,...,2,2,2,2,3,3,2,2,No,Yes
4,< 25 years,Female,Single,College/university,Other,Other,2,2,3,3,...,2,3,3,3,3,4,3,3,Yes,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281,41 - 55 years,Male,Married,Secondary/High school,Other,Other,2,2,2,2,...,2,2,2,2,2,2,2,2,Yes,Yes
282,above 55 years,Male,Married,Graduate school,Education,Top management,2,2,2,2,...,2,2,2,2,2,2,2,2,Yes,Yes
283,above 55 years,Female,Married,College/university,Other,Other,2,2,2,2,...,2,2,2,2,2,2,2,2,Yes,Yes
284,41 - 55 years,Male,Married,Graduate school,Education,Professional,3,3,3,3,...,3,3,3,3,3,3,3,3,Yes,Yes


## Get Unique Variables

In [10]:
def df_unique(arr):
    tmp_keys = []
    for val in arr:
        if type(val) != str:
            continue
        tmp_list = val.split(";")
        for val2 in tmp_list:
            if val2 not in tmp_keys:
                tmp_keys.append(val2)
                
    arr_list = []
    for val in arr:
        if type(val) != str:
            continue
        tmp_dict = {i:0 for i in tmp_keys}
        tmp_list = val.split(";")
        for val2 in tmp_list:
            tmp_dict[val2] += 1
        arr_list.append(tmp_dict)
    
    return pd.DataFrame(arr_list)

## Target Variable

In [11]:
# Allow Researcher to Select Which Unique Value to Filter By

#### DataFrame for Variables to Predict Against

In [31]:
tmp_y_df = pd.DataFrame()

Do you own any of the following? (Own Electronic Payment)

In [32]:
tmp_col = df.iloc[:, 12]
tmp_df = df_unique(tmp_col)

tmp_df.iloc[:, 2] = pd.concat([tmp_df.iloc[:, 2], tmp_df.iloc[:, -1]], axis = 1).max(axis=1)

# Convert tmp_df to Sum
tmp_dict = {col : tmp_df[col].sum() for col in tmp_df.columns[:-1]}

for ind in range(tmp_df.shape[1] - 1):
    tmp_col = tmp_df.iloc[:, ind]
    tmp_y_df[tmp_col.name] = tmp_col

name_arr = tmp_df.columns.tolist()[:-1]
name_arr[2] = "Touch n Go, EzLink etc"

tmp_df = pd.DataFrame(pd.Series(tmp_dict), columns = ["Yes"])
tmp_df["No"] = 286 - tmp_df["Yes"]
tmp_df = tmp_df.reset_index(drop = True).T
tmp_df.columns = [f"Epy{ind + 1}" for ind in range(tmp_df.shape[1])]

tmp_df = tmp_df / 286 * 100.0
fig = show_bar_graph_percentage(tmp_df, "", "Types of E-Payment", "Legend", "Percentage")
fig.show()

name_df = pd.DataFrame({}, index = [f"Epy{ind + 1}" for ind in range(len(name_arr))])
name_df["Name"] = name_arr

display(name_df.iloc[:, :].T)

,Epy1,Epy2,Epy3,Epy4,Epy5,Epy6,Epy7,Epy8
Name,Mobile Smartphone,"Bank Cards (Credit, Debit, Pre-paid)","Touch n Go, EzLink etc",Internet Services (e.g: Broadband),E-wallet account (E.g: MOL or PayPal),Internet of Things gadget (e.g: Fitbit – measu...,Blockchain / Cryptocurrency Coin,HealthCare Gadget (E.g: Blood pressure measure...


Have you made any electronic payments in the past 12 months? (Type of Electronic Payment)

In [33]:
tmp_col = df.iloc[:, 14]
tmp_df = df_unique(tmp_col)

tmp_df.iloc[:, 2] = pd.concat([tmp_df.iloc[:, 2], tmp_df.iloc[:, -1]], axis = 1).max(axis=1)

for ind in range(tmp_df.shape[1] - 2):
    tmp_col = tmp_df.iloc[:, ind]
    tmp_y_df[tmp_col.name] = tmp_col

# Convert tmp_df to Sum
tmp_dict = {col : tmp_df[col].sum() for col in tmp_df.columns[:-2]}

name_arr = tmp_df.columns.tolist()[:-2]
name_arr[2] = "Yes (Touch n Go, EzLink etc)"

tmp_df = pd.DataFrame(pd.Series(tmp_dict), columns = ["Yes"])
tmp_df["No"] = 286 - tmp_df["Yes"]
tmp_df = tmp_df.reset_index(drop = True).T
tmp_df.columns = [f"Epy{ind + 1}" for ind in range(tmp_df.shape[1])]

tmp_df = tmp_df / 286 * 100.0
fig = show_bar_graph_percentage(tmp_df, "History of E-Payment", "Types of E-Payment", "Legend", "Percentage")
fig.show()

name_df = pd.DataFrame({}, index = [f"Epy{ind + 1}" for ind in range(len(name_arr))])
name_df["Name"] = name_arr

display(name_df.iloc[:, :].T)

,Epy1,Epy2,Epy3,Epy4,Epy5,Epy6
Name,Yes (Mobile),"Yes (Bank Cards (i.e. Credit, Debit, Pre-paid))","Yes (Touch n Go, EzLink etc)",Yes (Internet Services (e.g: Broadband)),Yes (E-wallet account (E.g: MOL or PayPal)),Yes (Blockchain / Cryptocurrency Coin solutions)


In [34]:
tmp_col = df.iloc[:, 14]
tmp_df = df_unique(tmp_col)
tmp_dict = {"Yes" : 286 - tmp_df["No"].sum(), "No" : tmp_df["No"].sum()}

tmp_s = tmp_df["No"]

tmp_s = tmp_s.map(lambda x : 1 if x == 0 else 0)
tmp_s = tmp_s.rename("Have you ever used E-Payment Before?")
# Store into tmp_y_df
tmp_y_df[tmp_s.name] = tmp_s

tmp_df = pd.DataFrame(pd.Series(tmp_dict)).T
tmp_df.iloc[0, :] = tmp_df.iloc[0, :] / tmp_df.iloc[0, :].sum() * 100.0
show_bar_graph_percentage(tmp_df, "", "X Axis", "Y Axis", "Percentage")

Have you ever purchased anything using the E-payment mode?

In [35]:
tmp_col = df.iloc[:, 17]

# Rename Column
tmp_col = tmp_col.rename(tmp_col.name.split("[")[1][4:-1])

# Change "Yes" => 1, "No" => 0
tmp_col = tmp_col.map(lambda x : 1 if x == "Yes" else 0)

# Change Data Type to Int
tmp_col = tmp_col.astype(int)

# Store into tmp_y_df
tmp_y_df[tmp_col.name] = tmp_col

In [36]:
tmp_dict = {"Yes" : tmp_col.sum(), "No" : 286 - tmp_col.sum()}

tmp_df = pd.DataFrame(pd.Series(tmp_dict)).T
tmp_df.iloc[0, :] = tmp_df.iloc[0, :] / tmp_df.iloc[0, :].sum() * 100.0
show_bar_graph_percentage(tmp_df, "", "History of E-Payment?", "Y Axis", "Percentage")

In the next six months, do you plan to purchase anything using the E-payment mode?

In [37]:
tmp_col = df.iloc[:, 18]

# Rename Column
tmp_col = tmp_col.rename(tmp_col.name.split("[")[1][4:-1])

# Change "Yes" => 1, "No" => 0
tmp_col = tmp_col.map(lambda x : 1 if x == "Yes" else 0)

# Change Data Type to Int
tmp_col = tmp_col.astype(int)

# Store into tmp_y_df
tmp_y_df[tmp_col.name] = tmp_col

In [38]:
tmp_dict = {"Yes" : tmp_col.sum(), "No" : 286 - tmp_col.sum()}

tmp_df = pd.DataFrame(pd.Series(tmp_dict)).T
tmp_df.iloc[0, :] = tmp_df.iloc[0, :] / tmp_df.iloc[0, :].sum() * 100.0
show_bar_graph_percentage(tmp_df, "", "Intention to Use E-Payment?", "Y Axis", "Percentage")

## Data Preprocessing

In [40]:
arr_df = [mod_fac_df, utaut_fac_df]

df_X = pd.concat(arr_df, axis = 1)
df_Y = tmp_y_df.loc[:, "Have you ever purchased anything using the E-payment mode?"]

# Convert Values to Int
df_X = df_X.astype(int)
df_Y = df_Y.astype(int)

#### Moderated Variables

In [41]:
# # Use PointBiserialR as Y only has 2 classes
# arr_list = []

# prob = 0.95

# for col in mod_fac_df.columns:
#     chi_df = pd.crosstab(df_X.loc[:, col], df_Y)
#     stat, p, dof, expected = chi2_contingency(chi_df)
        
#     critical = chi2.ppf(prob, dof)
        
#     if abs(stat) >= critical:
#         arr_list.append((col, stat))
        
# tmp_df = pd.DataFrame(arr_list, columns = ["Moderated Variable", "Correlation Value"])

# tmp_df.style.hide_index()

#### Remove Features that are not significant with Target Variable

In [42]:
arr_list = []

prob = 0.95

# Get List of P_Values
for col in df_X.columns:
    chi_df = pd.crosstab(df_X.loc[:, col], df_Y)
    stat, p, dof, expected = chi2_contingency(chi_df)
    
    critical = chi2.ppf(prob, dof)
    
    alpha = 1 - prob
        
    if abs(stat) >= critical and p <= alpha:
        arr_list.append((col, stat))
        
# Sort Variables Ascending by P_val
arr_list = sorted(arr_list, key = lambda x : x[1])

arr_df = pd.DataFrame(arr_list, columns = ["Variables", "Chi-Square Value"])

arr_df.style.hide_index()

Variables,Chi-Square Value
FC4,9.553856
BI1,10.510160
T4,10.692050
T1,11.540024
SE3,12.217458
PE3,14.351065
Education Level,23.811479


### Merit Based Ranking

In [43]:
arr_list = []

tmp_X = df_X
tmp_Y = df_Y

for col_ind in range(tmp_X.shape[1]):
    name = tmp_X.columns[col_ind]
    merit = merit_calculation(tmp_X.iloc[:, [col_ind]], tmp_Y, pointbiserialr)
    arr_list.append((col_ind ,name, round(merit, 4)))
    
# Sort Column By Merit Value
arr_list = sorted(arr_list, key = lambda x : x[2], reverse = True)

mbf_df = pd.DataFrame(arr_list, index = [i + 1 for i in range(len(arr_list))], columns = ["Rank" ,"Variables", "Merit"])

mbf_df["Rank"] = [ind + 1 for ind in range(mbf_df.shape[0])]

mbf_df.style.hide_index()

Rank,Variables,Merit
1,Education Level,0.198100
2,T4,0.174500
3,BI1,0.170200
4,SE4,0.146800
5,T2,0.138300
6,SE3,0.130500
7,SE2,0.128400
8,T3,0.128300
9,FC4,0.126600
10,AT1,0.122900


### CFS

In [46]:
arr_df = [utaut_fac_df]

tmp_X = pd.concat(arr_df, axis = 1)
tmp_Y = df_Y

feature_set = select_by_CFS(tmp_X, tmp_Y)

feature_set

{'AX2', 'AX4', 'BI1', 'FC4', 'SE4', 'T4'}

In [47]:
corr_df = create_corr_metric_matrix(df_X.loc[:, feature_set], pearsonr)

corr_df = corr_df.where(np.tril(np.ones(corr_df.shape)).astype(np.bool))

corr_df

,FC4,T4,BI1,AX2,SE4,AX4
FC4,1.000000,NaN,NaN,NaN,NaN,NaN
T4,0.609449,1.000000,NaN,NaN,NaN,NaN
BI1,0.645913,0.698777,1.000000,NaN,NaN,NaN
AX2,0.014927,0.126601,0.021935,1.000000,NaN,NaN
SE4,0.545745,0.657036,0.655867,0.074875,1.000000,NaN
AX4,0.091958,0.015776,0.121645,0.672771,0.042893,1.0


### Model Object

In [48]:
class ModelObj:
    def __init__(self, model, name, accuracy, clf_report, confusion_matrix, mcc, time_taken):
        self.model = model
        self.name = name
        self.accuracy = accuracy
        self.clf_report = clf_report
        self.confusion_matrix = confusion_matrix
        self.mcc = mcc
        self.time_taken = time_taken

In [49]:
def create_ModelObj(model, name, X, Y, class_arr):
    
    # Time Taken
    start = time.process_time()
    
    # Train Test Split5
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3)
    
    # Train Model
    model.fit(X_train, y_train)
    
    # Get Y Predict
    y_pred = model.predict(X_test)
    
    # Accuracy
    acc_score = get_acc_score_kcv(X_train, y_train, model)

    # Classification Report
    tf_dict = { str(ind):val for ind, val in enumerate(class_arr)}
    clf_report = pd.DataFrame(classification_report(y_test, y_pred, output_dict = True))
    clf_report.rename(tf_dict, axis = 1, inplace=True)
    clf_report = clf_report.T

    # Confusion Matrix
    tf_dict = { ind:val for ind, val in enumerate(class_arr)}
    confusion_matrix_df = pd.DataFrame(confusion_matrix(y_test, y_pred))
    confusion_matrix_df.rename(tf_dict, axis = 0, inplace=True)
    confusion_matrix_df.rename(tf_dict, axis = 1, inplace=True)
    
    # Matthew Correlation Coefficient
    mcc = matthews_corrcoef(y_test, y_pred)
    
    # your code here 
    time_taken = time.process_time() - start
    
    return ModelObj(model, name, acc_score, clf_report, confusion_matrix_df, mcc, time_taken)

In [50]:
model_dict = {}

### Decision Tree

In [51]:
tmp_X = df_X
tmp_Y = df_Y

model = DecisionTreeClassifier()
name = "Decision Tree"

acc_score = get_acc_score_kcv(tmp_X, tmp_Y, model)

print(f"Accuracy Score: {round(acc_score, 2)}%")

model_dict[name] = create_ModelObj(model, name, tmp_X, tmp_Y, ["Disagree", "Agree"])

Accuracy Score: 74.86%


In [52]:
tmp_X = df_X.loc[:, feature_set]
tmp_Y = df_Y

model = DecisionTreeClassifier()
name = "Decision Tree"

acc_score = get_acc_score_kcv(tmp_X, tmp_Y, model)

print(f"Accuracy Score: {round(acc_score, 2)}%")

model_dict[name] = create_ModelObj(model, name, tmp_X, tmp_Y, ["Disagree", "Agree"])

Accuracy Score: 80.76%


### Random Forest (Bagging Method)

In [53]:
tmp_X = df_X
tmp_Y = df_Y

model = RandomForestClassifier()
name = "Random Forest"

acc_score = get_acc_score_kcv(tmp_X, tmp_Y, model)

print(f"Accuracy Score: {round(acc_score, 2)}%")

model_dict[name] = create_ModelObj(model, name, tmp_X, tmp_Y, ["Disagree", "Agree"])

Accuracy Score: 85.33%


In [54]:
tmp_X = df_X.loc[:, feature_set]
tmp_Y = df_Y

model = RandomForestClassifier()
name = "Random Forest"

acc_score = get_acc_score_kcv(tmp_X, tmp_Y, model)

print(f"Accuracy Score: {round(acc_score, 2)}%")

model_dict[name] = create_ModelObj(model, name, tmp_X, tmp_Y, ["Disagree", "Agree"])

Accuracy Score: 87.41%


### XGBoost (Boosting Method)

In [55]:
tmp_X = df_X
tmp_Y = df_Y

model = XGBClassifier(eval_metric='error', use_label_encoder=False)
name = "XGBoost"

acc_score = get_acc_score_kcv(tmp_X, tmp_Y, model)

print(f"Accuracy Score: {round(acc_score, 2)}%")

model_dict[name] = create_ModelObj(model, name, tmp_X, tmp_Y, ["Disagree", "Agree"])

Accuracy Score: 86.75%


In [56]:
tmp_X = df_X.loc[:, feature_set]
tmp_Y = df_Y

model = XGBClassifier(eval_metric='error', use_label_encoder=False)
name = "XGBoost"

acc_score = get_acc_score_kcv(tmp_X, tmp_Y, model)

print(f"Accuracy Score: {round(acc_score, 2)}%")

model_dict[name] = create_ModelObj(model, name, tmp_X, tmp_Y, ["Disagree", "Agree"])

Accuracy Score: 86.02%


## Plot Graph

### Result

In [57]:
m_arr = [(name, model_dict[name].clf_report, model_dict[name].mcc, model_dict[name].time_taken) for name in model_dict]
cmp_result_tbl(m_arr, "weighted avg")

,Precision,Recall,F1-Score,MCC,Time Taken
Decision Tree,0.750662,0.790698,0.770160,-0.113999,0.046875
Random Forest,0.819557,0.872093,0.845009,-0.060886,1.390625
XGBoost,0.797079,0.848837,0.822144,-0.075509,7.828125


### Precision

In [58]:
clf_report_arr = [(name, model_dict[name].clf_report) for name in model_dict]
tmp_df = get_df_type(clf_report_arr, "Precision")
pfr_graph(tmp_df, "Model", "Score", "Precison Comparison")

### Recall

In [59]:
clf_report_arr = [(name, model_dict[name].clf_report) for name in model_dict]
tmp_df = get_df_type(clf_report_arr, "Recall")
pfr_graph(tmp_df, "Model", "Score", "Recall Comparison")

### F1-Score

In [60]:
clf_report_arr = [(name, model_dict[name].clf_report) for name in model_dict]
tmp_df = get_df_type(clf_report_arr, "F1-Score")
pfr_graph(tmp_df, "Model", "Score", "F1-Score Comparison")

### Accuracy

In [61]:
acc_arr = [(key, model_dict[key].accuracy) for key in model_dict]
acc_graph(acc_arr, "Accuracy Score Comparison", "Accuracy Score", "Types of Model")

In [62]:
pd.DataFrame(acc_arr)

,0,1
0,Decision Tree,79.5
1,Random Forest,85.5
2,XGBoost,84.0


### Stepwise Feature Selection

In [194]:
tmp_X = df_X.loc[:, mbf_df["Variables"]]
tmp_Y = df_Y

model = DecisionTreeClassifier()

ft_df = cmp_n_feature_df(tmp_X, tmp_Y, model, forward_selection, 3, 13)

In [150]:
ft_df.style.hide_index()

Number of Features,Feature Set,Accuracy Score
3,"['T4', 'BI4', 'AT2']",93.370000
4,"['T1', 'Gender', 'BI3', 'SI3']",92.360000
5,"['T1', 'EE4', 'SE2', 'T2', 'SI3']",93.730000
6,"['PE1', 'T1', 'AT3', 'SE4', 'EE2', 'SE1']",92.320000
7,"['PE2', 'PE1', 'Gender', 'EE3', 'PE3', 'Age', 'SI3']",91.970000
8,"['EE4', 'Work Position', 'EE3', 'AT3', 'SE4', 'SI1', 'SI3', 'SI4']",92.270000
9,"['Education Level', 'PE1', 'PE4', 'Work Industry', 'PE3', 'T2', 'SE3', 'AT2', 'T3']",92.290000
10,"['PE2', 'PE1', 'EE1', 'SE2', 'EE3', 'PE3', 'SE4', 'T2', 'FC2', 'T3']",92.330000
11,"['PE2', 'PE1', 'T1', 'Gender', 'PE3', 'AT4', 'Age', 'FC2', 'FC1', 'SE1', 'SI2']",92.300000
12,"['T1', 'SE2', 'Work Position', 'T4', 'AT3', 'BI3', 'T2', 'SE3', 'FC1', 'AX3', 'AX2', 'FC3']",91.980000


In [124]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = ft_df["Number of Features"], y = ft_df["Accuracy Score"]))
fig.update_layout(
    title = "Accuracies based on Selected Features"
)
fig.show()